In [1]:
#Segmenting and Clustering Neighborhoods in Toronto Part 2

#import libraries
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

In [4]:
#Read in the data & print the headers
df_can = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df_can.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
#clean the data:
#   - ignore cells with Borough = "Not assigned"
#   - if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_can = df_can[df_can['Borough'] != 'Not assigned']
df_can.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
#check if there are any neighborhoods that are still "Not Assigned"
df_can2 = df_can[df_can['Neighbourhood'] == 'Not assigned']
df_can2.head()
#there are none so no need to reassign them to match the Borough

,Postal Code,Borough,Neighbourhood


In [14]:
#use the .shape method to print the number of rows of your dataframe
np.shape(df_can)

(103, 3)

In [15]:
#download geographical coordinates of the neighborhoods in the Toronto. 
df_geo = pd.read_csv('http://cocl.us/Geospatial_data', index_col=0)

print('Dataset downloaded and read into a pandas dataframe!')

df_geo.head()

Dataset downloaded and read into a pandas dataframe!


,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [17]:
#merge the 2 datasets together
df_full = df_can.merge(df_geo, on='Postal Code')
df_full.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [21]:
#pull out only the Boroughs that contain Toronto in the name
df_toronto = df_full[df_full.Borough.str.contains('Toronto',case=False)]
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [22]:
#import additional libraries

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [32]:
# create map of Toronto using latitude and longitude values
latitude = 43.65
longitude = -79.38

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto